In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler

In [2]:
device = torch.device('cpu')

In [3]:
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
#test_batch_size = 10000
learning_rate = 0.001
#SPC is for number of samples per class
t1=[]
t2=[]
t3=[]
i=0

In [4]:
# MNIST dataset 

train_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]),  
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]))


t0=test_dataset.targets
t0 = t0.reshape(1, -1)
t0 = t0.squeeze()
t2=t0.tolist()
print ( "Test list size is" ,len(t2))

Test list size is 10000


In [5]:
# Data loader


train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size ,shuffle=False)

print (len(test_loader))
print (len(test_dataset))



100
10000


In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = torch.nn.Linear(input_size, hidden_size) 
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [9]:
# Train the model for all elements of the dataset
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/600], Loss: 0.4685
Epoch [1/2], Step [200/600], Loss: 0.5386
Epoch [1/2], Step [300/600], Loss: 0.4100
Epoch [1/2], Step [400/600], Loss: 0.5052
Epoch [1/2], Step [500/600], Loss: 0.5533
Epoch [1/2], Step [600/600], Loss: 0.3194
Epoch [2/2], Step [100/600], Loss: 0.2918
Epoch [2/2], Step [200/600], Loss: 0.4281
Epoch [2/2], Step [300/600], Loss: 0.3290
Epoch [2/2], Step [400/600], Loss: 0.4691
Epoch [2/2], Step [500/600], Loss: 0.5069
Epoch [2/2], Step [600/600], Loss: 0.3003


In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
t3=[]
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        tpredict=predicted.tolist()
        t3=t3+tpredict
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        print("Current total samples are",n_samples,)
        print("of which ",n_correct,"samples are correct")

        
print(len(t3))
       
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network on the given test images: {acc} %')


Current total samples are 100
of which  87 samples are correct
Current total samples are 200
of which  176 samples are correct
Current total samples are 300
of which  262 samples are correct
Current total samples are 400
of which  347 samples are correct
Current total samples are 500
of which  433 samples are correct
Current total samples are 600
of which  517 samples are correct
Current total samples are 700
of which  603 samples are correct
Current total samples are 800
of which  695 samples are correct
Current total samples are 900
of which  785 samples are correct
Current total samples are 1000
of which  862 samples are correct
Current total samples are 1100
of which  951 samples are correct
Current total samples are 1200
of which  1035 samples are correct
Current total samples are 1300
of which  1121 samples are correct
Current total samples are 1400
of which  1205 samples are correct
Current total samples are 1500
of which  1294 samples are correct
Current total samples are 1600


In [11]:
from sklearn.metrics import classification_report
#print(classification_report(labels, predicted, zero_division=1))
print(classification_report(t2, t3, zero_division=1))

              precision    recall  f1-score   support

           0       0.76      0.88      0.82      1000
           1       0.97      0.97      0.97      1000
           2       0.80      0.76      0.78      1000
           3       0.85      0.89      0.87      1000
           4       0.80      0.73      0.77      1000
           5       0.96      0.90      0.93      1000
           6       0.67      0.60      0.63      1000
           7       0.88      0.96      0.92      1000
           8       0.94      0.96      0.95      1000
           9       0.95      0.94      0.95      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [12]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(t2,t3))

[[880   0  11  29   3   6  56   1  14   0]
 [  4 970   0  20   2   0   2   0   2   0]
 [ 22   1 764  15 102   1  89   1   5   0]
 [ 37  23   6 887  21   1  21   0   4   0]
 [  2   2  84  51 733   0 123   0   5   0]
 [  0   0   0   1   0 897   0  78   2  22]
 [201   2  86  29  54   2 599   0  27   0]
 [  0   0   0   0   0  12   0 960   0  28]
 [  4   0   5   8   1  12   3   7 960   0]
 [  1   0   0   0   0   5   0  50   0 944]]
